In [1]:
import numpy as np
pi=np.pi
def sin(t):
    return(np.round(np.sin(t/180*pi),9))
def cos(t):
    return(np.round(np.cos(t/180*pi),9))
def ten(t):
    return(np.round(np.tan(t/180*pi),9))
def atan2(x,y):
    return(round(np.arctan2(x,y)*180/pi,5))
def P3(x,y,z):#位置矢量
    return(np.mat([[x],[y],[z]]))
def P4(x,y,z):#位置矢量
    return(np.mat([[x],[y],[z],[1]]))
def DQ(x,y,z):#平移算式
    return(np.mat([[1,0,0,x],
                     [0,1,0,y],
                     [0,0,1,z],
                     [0,0,0,1]],dtype='float'))
def Rx(t):#X旋轉
    return(np.mat([[1,0,0],
                     [0,cos(t),-sin(t)],
                     [0,sin(t),cos(t)]]))
def Ry(t):#Y旋轉
    return(np.mat([[cos(t),0,sin(t)],
                     [0,1,0],
                     [-sin(t),0,cos(t)]]))
def Rz(t):#Z旋轉
    return(np.mat([[cos(t),-sin(t),0],
                     [sin(t),cos(t),0],
                     [0,0,1]]))  
def R(r11,r12,r13,r21,r22,r23,r31,r32,r33):#平移算式
    return(np.mat([[r11,r12,r13],
                     [r21,r22,r23],
                     [r31,r32,r33]]))
def T(r,dq):#一般變換!!!
    dq[0:3,0:3]=r
    return(dq)
def Tt(r,p):#逆變換!!!
    p=-r.T*p
    return(np.vstack((np.hstack((r.T,p)),[0,0,0,1])))
def Rxyz(r,b,a):#XYZ固定角
    return(np.mat([[cos(a)*cos(b),cos(a)*sin(b)*sin(r)-sin(a)*cos(r),cos(a)*sin(b)*cos(r)+sin(a)*sin(r)],
                     [sin(a)*cos(b),sin(a)*sin(b)*sin(r)+cos(a)*cos(r),sin(a)*sin(b)*cos(r)-cos(a)*sin(r)],
                     [-sin(b),cos(b)*sin(r),cos(b)*cos(r)]]))
def Rzyz(r,b,a):#ZYZ歐拉角
    return(np.mat([[cos(a)*cos(b)*cos(r)-sin(a)*sin(r),-cos(a)*cos(b)*sin(r)-sin(a)*cos(r),cos(a)*sin(b)],
                     [sin(a)*cos(b)*cos(r)+cos(a)*sin(r),-sin(a)*cos(b)*sin(r)+cos(a)*cos(r),sin(a)*sin(b)],
                     [-sin(b)*cos(r),sin(b)*sin(r),cos(b)]]))                     
def Rk(th,x,y,z):#等效角
    return(np.mat([[x*x*(1-cos(th))+cos(th),y*x*(1-cos(th))-z*sin(th),z*x*(1-cos(th))+y*sin(th)],
                     [x*y*(1-cos(th))+z*sin(th),y*y*(1-cos(th))+cos(th),z*y*(1-cos(th))-x*sin(th)],
                     [x*z*(1-cos(th))-y*sin(th),y*z*(1-cos(th))+x*sin(th),z*z*(1-cos(th))+cos(th)]]))
def DH(i,a,alpha,d,theta):
    i-=1
    return(np.mat([[cos(theta[i]),-sin(theta[i]),0,a[i]],
                     [sin(theta[i])*cos(alpha[i]),cos(theta[i])*cos(alpha[i]),-sin(alpha[i]),-sin(alpha[i])*d[i]],
                     [sin(theta[i])*sin(alpha[i]),cos(theta[i])*sin(alpha[i]),cos(alpha[i]),cos(alpha[i])*d[i]],
                     [0,0,0,1]]))


In [3]:
h01=DQ(0,100,0)*DQ(0,0,100)
h12=DQ(-50,0,0)*DQ(0,50,0)
h23=DQ(0,0,200)*T(Rz(90),DQ(0,0,0))*T(Rx(180),DQ(0,0,0))
h03=h01*h12*h23
print(h01)
print(h12)
print(h23)
print(h03)

[[  1.   0.   0.   0.]
 [  0.   1.   0. 100.]
 [  0.   0.   1. 100.]
 [  0.   0.   0.   1.]]
[[  1.   0.   0. -50.]
 [  0.   1.   0.  50.]
 [  0.   0.   1.   0.]
 [  0.   0.   0.   1.]]
[[  0.   1.   0.   0.]
 [  1.   0.   0.   0.]
 [  0.   0.  -1. 200.]
 [  0.   0.   0.   1.]]
[[  0.   1.   0. -50.]
 [  1.   0.   0. 150.]
 [  0.   0.  -1. 300.]
 [  0.   0.   0.   1.]]
